# Logistic Regression in scikit-learn


## Introduction

Generally, the process for fitting a logistic regression model using scikit-learn is very similar to that which you previously saw for `statsmodels`. One important exception is that scikit-learn will not display statistical measures such as the p-values associated with the various features. This is a shortcoming of scikit-learn, although scikit-learn has other useful tools for tuning models which we will investigate in future lessons.

The other main process of model building and evaluation which we didn't discuss previously is performing a train-test split. As we saw in linear regression, model validation is an essential part of model building as it helps determine how our model will generalize to future unseen cases. After all, the point of any model is to provide future predictions where we don't already know the answer but have other informative data (`X`).

With that, let's take a look at implementing logistic regression in scikit-learn using dummy variables and a proper train-test split.


## Objectives

You will be able to:

- Fit a logistic regression model using scikit-learn 

## Importing the Data

In [1]:
import pandas as pd

df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Defining `X` and `y`

To start out, we'll consider `y` to be the target variable (`Survived`) and everything else to be `X`.

In [2]:
y = df["Survived"]
X = df.drop("Survived", axis=1)

## Train-Test Split

Specifying a `random_state` means that we will get consistent results even if the kernel is restarted.

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## Preprocessing

### Dealing with Missing Data

Some of the data is missing, which won't work with a scikit-learn model:

In [4]:
X_train.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            133
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          511
Embarked         2
dtype: int64

For `Cabin` and `Embarked` (categorical features), we'll manually fill this in with "missing" labels:

In [5]:
X_train_fill_na = X_train.copy()
X_train_fill_na.fillna({"Cabin":"cabin_missing", "Embarked":"embarked_missing"}, inplace=True)
X_train_fill_na.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            133
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
dtype: int64

For `Age` (a numeric feature), we'll use a `SimpleImputer` from scikit-learn ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)) to fill in the mean:

In [6]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()

imputer.fit(X_train_fill_na[["Age"]])
age_imputed = pd.DataFrame(
    imputer.transform(X_train_fill_na[["Age"]]),
    # index is important to ensure we can concatenate with other columns
    index=X_train_fill_na.index,
    columns=["Age"]
)

X_train_fill_na["Age"] = age_imputed
X_train_fill_na.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### Dealing with Categorical Data

Some of the columns of `X_train_fill_na` currently contain categorical data (i.e. Dtype `object`):

In [7]:
X_train_fill_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668 entries, 105 to 684
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  668 non-null    int64  
 1   Pclass       668 non-null    int64  
 2   Name         668 non-null    object 
 3   Sex          668 non-null    object 
 4   Age          668 non-null    float64
 5   SibSp        668 non-null    int64  
 6   Parch        668 non-null    int64  
 7   Ticket       668 non-null    object 
 8   Fare         668 non-null    float64
 9   Cabin        668 non-null    object 
 10  Embarked     668 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 62.6+ KB


In [8]:
X_train_categorical = X_train_fill_na.select_dtypes(exclude=["int64", "float64"]).copy()
X_train_categorical

,Name,Sex,Ticket,Cabin,Embarked
105,"Mionoff, Mr. Stoytcho",male,349207,cabin_missing,S
68,"Andersson, Miss. Erna Alexandra",female,3101281,cabin_missing,S
253,"Lobb, Mr. William Arthur",male,A/5. 3336,cabin_missing,S
320,"Dennis, Mr. Samuel",male,A/5 21172,cabin_missing,S
706,"Kelly, Mrs. Florence ""Fannie""",female,223596,cabin_missing,S
...,...,...,...,...,...
835,"Compton, Miss. Sara Rebecca",female,PC 17756,E49,C
192,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,350046,cabin_missing,S
629,"O'Connell, Mr. Patrick D",male,334912,cabin_missing,Q
559,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",female,345572,cabin_missing,S


In [9]:
# Inspect unique categories in each categorical variable
for column in X_train_categorical.columns:
    unique_categories = X_train_categorical[column].unique()
    print(f"Unique categories in {column}:")
    print(unique_categories)

    # You can also use value_counts to see the frequency of each category
    category_counts = X_train_categorical[column].value_counts()
    print(f"Category counts in {column}:")
    print(category_counts)



Unique categories in Name:
['Mionoff, Mr. Stoytcho' 'Andersson, Miss. Erna Alexandra'
 'Lobb, Mr. William Arthur' 'Dennis, Mr. Samuel'
 'Kelly, Mrs. Florence "Fannie"' 'Tornquist, Mr. William Henry'
 'Rosblom, Mr. Viktor Richard' 'Vande Velde, Mr. Johannes Joseph'
 'Herman, Miss. Alice' 'Heikkinen, Miss. Laina'
 'Dahlberg, Miss. Gerda Ulrika' 'Smart, Mr. John Montgomery'
 'Hakkarainen, Mr. Pekka Pietari' 'Ward, Miss. Anna'
 'Harper, Miss. Annie Jessie "Nina"'
 'Mellinger, Mrs. (Elizabeth Anne Maidment)' 'Olsen, Mr. Ole Martin'
 'Saad, Mr. Amin' 'Hays, Mrs. Charles Melville (Clara Jennings Gregg)'
 'Tikkanen, Mr. Juho' 'Fynney, Mr. Joseph J' 'Taussig, Miss. Ruth'
 'Moubarek, Master. Gerios' 'Peters, Miss. Katie' 'Laleff, Mr. Kristo'
 'Williams, Mr. Howard Hugh "Harry"' 'Moor, Mrs. (Beila)'
 'Vovk, Mr. Janko' 'Vander Planke, Mr. Leo Edmondus'
 'Thayer, Mr. John Borland' 'Frost, Mr. Anthony Wood "Archie"'
 'Goodwin, Miss. Lillian Amy'
 'Arnold-Franchi, Mrs. Josef (Josefine Franchi)' 'Kell

In [10]:
# Handle missing values in categorical variables using methods like imputation or removal
# For example, you can use SimpleImputer from scikit-learn for imputation
from sklearn.impute import SimpleImputer

# Create an instance of SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')

# Fit the imputer to X_train_categorical and transform it
X_train_categorical_imputed = imputer.fit_transform(X_train_categorical)

# Ensure that all categorical variables are properly encoded (e.g., as strings or categorical data types)
# Convert categorical variables to appropriate data types if necessary
# For example, convert numeric variables to strings
X_train_categorical_imputed = X_train_categorical_imputed.astype(str)


`OneHotEncoder` from scikit-learn ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)) can be used to convert categorical variables into dummy one-hot encoded variables:

In [11]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ohe = OneHotEncoder(handle_unknown="ignore")

# Assuming X_train_categorical is your input data
X_train_ohe = ohe.fit_transform(X_train_categorical)

# Get the shape of the transformed data
transformed_shape = X_train_ohe.shape

# Print the shape
print("Shape of transformed data:", transformed_shape)


Shape of transformed data: (668, 1336)


In [12]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False) #removed sparse_output=False from parentheses

ohe.fit(X_train_categorical)
X_train_ohe = pd.DataFrame(
    ohe.transform(X_train_categorical),
    # index is important to ensure we can concatenate with other columns
    index=X_train_categorical.index,
    # we are dummying multiple columns at once, so stack the names
    columns=np.hstack(ohe.categories_)
)
X_train_ohe

TypeError: __init__() got an unexpected keyword argument 'sparse_output'

In [ ]:
# After transforming the data with OneHotEncoder, inspect the resulting column names
# Use the get_feature_names_out() method of the OneHotEncoder object to retrieve the names of the encoded features
encoded_feature_names = ohe.get_feature_names_out(input_features=X_train_categorical.columns)

# Verify that the column names match the transformed data and that there are no discrepancies in the alignment
# Print the encoded feature names
print("Encoded feature names:")
print(encoded_feature_names)


Wow! That's a lot of columns! Way more than is useful in practice: we now have columns for each of the passenger's names. This is an example of what not to do. Let's try that again, this time being mindful of which variables we actually want to include in our model.

Instead of just selecting every single categorical feature for dummying, let's only include the ones that make sense as categories rather than being the names of individual people:

In [ ]:
categorical_features = ["Sex", "Cabin", "Embarked"]
X_train_categorical = X_train_fill_na[categorical_features].copy()
X_train_categorical

In [ ]:
ohe.fit(X_train_categorical)

X_train_ohe = pd.DataFrame(
    ohe.transform(X_train_categorical),
    index=X_train_categorical.index,
    columns=np.hstack(ohe.categories_)
)
X_train_ohe

That's still a lot of columns, but we no longer have more columns than records!

### Normalization

Now let's look at the numeric features. This time we'll also pay more attention to the meaning of the features, and only include relevant ones (e.g. not including `PassengerId` because this is a data artifact, not a true feature).

Another important data preparation practice is to normalize your data. That is, if the features are on different scales, some features may impact the model more heavily then others. To level the playing field, we often normalize all features to a consistent scale of 0 to 1.

As you can see, our features are currently not on a consistent scale:

In [ ]:
numeric_features = ["Pclass", "Age", "SibSp", "Fare"]
X_train_numeric = X_train_fill_na[numeric_features].copy()
X_train_numeric

Let's use a `MinMaxScaler` from scikit-learn ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)) with default parameters to create a maximum value of 1 and a minimum value of 0. This will work well with our binary one-hot encoded data.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(X_train_numeric)
X_train_scaled = pd.DataFrame(
    scaler.transform(X_train_numeric),
    # index is important to ensure we can concatenate with other columns
    index=X_train_numeric.index,
    columns=X_train_numeric.columns
)
X_train_scaled

Then we concatenate everything together:

In [ ]:
X_train_full = pd.concat([X_train_scaled, X_train_ohe], axis=1)
X_train_full

## Fitting a Model

Now let's fit a model to the preprocessed training set. In scikit-learn, you do this by first creating an instance of the `LogisticRegression` class. From there, then use the `.fit()` method from your class instance to fit a model to the training data.

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')
model_log = logreg.fit(X_train_full, y_train)
model_log

## Model Evaluation

Now that we have a model, lets take a look at how it performs.

### Performance on Training Data

First, how does it perform on the training data?

In the cell below, `0` means the prediction and the actual value matched, whereas `1` means the prediction and the actual value did not match.

In [ ]:
y_hat_train = logreg.predict(X_train_full)

train_residuals = np.abs(y_train - y_hat_train)
print(pd.Series(train_residuals, name="Residuals (counts)").value_counts())
print()
print(pd.Series(train_residuals, name="Residuals (proportions)").value_counts(normalize=True))

Not bad; our classifier was about 85% correct on our training data!

### Performance on Test Data

Now let's apply the same preprocessing process to our test data, so we can evaluate the model's performance on unseen data.

In [ ]:
# Filling in missing categorical data
X_test_fill_na = X_test.copy()
X_test_fill_na.fillna({"Cabin":"cabin_missing", "Embarked":"embarked_missing"}, inplace=True)

# Filling in missing numeric data
test_age_imputed = pd.DataFrame(
    imputer.transform(X_test_fill_na[["Age"]]),
    index=X_test_fill_na.index,
    columns=["Age"]
)
X_test_fill_na["Age"] = test_age_imputed

# Handling categorical data
X_test_categorical = X_test_fill_na[categorical_features].copy()
X_test_ohe = pd.DataFrame(
    ohe.transform(X_test_categorical),
    index=X_test_categorical.index,
    columns=np.hstack(ohe.categories_)
)

# Normalization
X_test_numeric = X_test_fill_na[numeric_features].copy()
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_numeric),
    index=X_test_numeric.index,
    columns=X_test_numeric.columns
)

# Concatenating categorical and numeric data
X_test_full = pd.concat([X_test_scaled, X_test_ohe], axis=1)
X_test_full

In [ ]:
y_hat_test = logreg.predict(X_test_full)

test_residuals = np.abs(y_test - y_hat_test)
print(pd.Series(test_residuals, name="Residuals (counts)").value_counts())
print()
print(pd.Series(test_residuals, name="Residuals (proportions)").value_counts(normalize=True))

And still about 78% accurate on our test data!

## Summary

In this lesson, you took a more complete look at a data science pipeline for logistic regression, splitting the data into training and test sets and using the model to make predictions. You'll practice this on your own in the upcoming lab before having a more detailed discussion of more nuanced methods for evaluating a classifier's performance.